<a href="https://colab.research.google.com/github/BrunoMaia23/Am/blob/main/MPV2_PUC_RJ_Probabilidade_Brasil_ser_campe%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Passo 1: Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

# Passo 2: Carregar e preparar os dados
url_raw = 'https://raw.githubusercontent.com/BrunoMaia23/Am/refs/heads/main/Fifa_world_cup_matches.csv'
df = pd.read_csv(url_raw)

# Criar a Coluna Alvo
df['Winner'] = df.apply(lambda row: 'team1' if row['number of goals team1'] > row['number of goals team2']
                                  else 'team2' if row['number of goals team1'] < row['number of goals team2']
                                  else 'draw', axis=1)

# Definir Variáveis Independentes e a Variável Alvo
coluna_alvo = 'Winner'
X = df.drop(columns=[coluna_alvo])
y = df[coluna_alvo]

# Identificar e separar as colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Criar LabelEncoders para variáveis categóricas
label_encoders = {}
for feature in categorical_features:
    if feature not in ['team1', 'team2']:  # Excluir 'team1' e 'team2' do encoding
        le = LabelEncoder()
        X[feature] = le.fit_transform(X[feature])
        label_encoders[feature] = le

# Garantir que 'team1' e 'team2' sejam strings e depois aplicar encoding
X['team1'] = X['team1'].astype(str)
X['team2'] = X['team2'].astype(str)
team1_le = LabelEncoder()
team2_le = LabelEncoder()
X['team1'] = team1_le.fit_transform(X['team1'])
X['team2'] = team2_le.fit_transform(X['team2'])

# Normalização dos dados numéricos
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Separar os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o Modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Avaliar o modelo
y_pred = modelo.predict(X_test)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

# Salvar o Modelo
with open('modelo_random_forest.pkl', 'wb') as file:
    pickle.dump(modelo, file)

print("Modelo salvo com sucesso em 'modelo_random_forest.pkl'")

# Passo 8: Fazer a Previsão com o Novo Modelo
# Carregar o modelo treinado
with open('modelo_random_forest.pkl', 'rb') as file:
    modelo = pickle.load(file)

# Criar dados de exemplo para o Brasil
dados_brasil = pd.DataFrame({
    'team1': ['Brazil'],
    'team2': ['Uruguay'],  # Substitua pelo nome do time adversário
    'number of goals team1': [0],  # Substitua pelos valores reais se disponíveis
    'number of goals team2': [0]    # Substitua pelos valores reais se disponíveis
})

# Garantir que todas as colunas esperadas estejam presentes
# Adicionar colunas ausentes com valores nulos ou padrão
for col in X.columns:
    if col not in dados_brasil.columns:
        dados_brasil[col] = 0  # Ou np.nan se apropriado

# Garantir que 'team1' e 'team2' sejam strings e depois aplicar encoding
dados_brasil['team1'] = dados_brasil['team1'].astype(str)
dados_brasil['team2'] = dados_brasil['team2'].astype(str)

# Checar e adicionar labels ausentes no LabelEncoder
# Para 'team1'
if not all(team in team1_le.classes_ for team in dados_brasil['team1']):
    missing_classes = set(dados_brasil['team1']) - set(team1_le.classes_)
    for missing_class in missing_classes:
        team1_le.classes_ = np.append(team1_le.classes_, missing_class)
        team1_le.classes_ = np.sort(team1_le.classes_)

# Para 'team2'
if not all(team in team2_le.classes_ for team in dados_brasil['team2']):
    missing_classes = set(dados_brasil['team2']) - set(team2_le.classes_)
    for missing_class in missing_classes:
        team2_le.classes_ = np.append(team2_le.classes_, missing_class)
        team2_le.classes_ = np.sort(team2_le.classes_)

# Aplicar o encoding
dados_brasil['team1'] = team1_le.transform(dados_brasil['team1'])
dados_brasil['team2'] = team2_le.transform(dados_brasil['team2'])

# Normalização dos dados (apenas nas colunas numéricas)
dados_brasil[numeric_features] = scaler.transform(dados_brasil[numeric_features])

# Garantir que as colunas estejam na mesma ordem que no modelo
dados_brasil = dados_brasil[X.columns]

# Fazer a previsão
probabilidade = modelo.predict_proba(dados_brasil)
probabilidade_vitoria = probabilidade[0][1]  # Probabilidade de vitória
print(f"A probabilidade do Brasil ser campeão da Copa do Mundo de 2026 é: {probabilidade_vitoria:.2%}")


Relatório de Classificação:
              precision    recall  f1-score   support

        draw       1.00      0.50      0.67         4
       team1       0.67      1.00      0.80         4
       team2       1.00      1.00      1.00         5

    accuracy                           0.85        13
   macro avg       0.89      0.83      0.82        13
weighted avg       0.90      0.85      0.84        13

Modelo salvo com sucesso em 'modelo_random_forest.pkl'
A probabilidade do Brasil ser campeão da Copa do Mundo de 2026 é: 20.00%
